# Import Libraries

In [4]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from google.colab import drive
from collections import Counter
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
!python -m spacy download en
import spacy
nlp = spacy.load('en')
spacy.prefer_gpu()

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


True

# Mount Google Drive

In [0]:
# Mount drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Import and Load Trained Model

In [0]:
# Copy checkpoint
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

# Start session and load model
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1', multi_gpu=True)

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


# Interact with the bot!

In [0]:
def interact_with_bot(moving_window=3, min_len_response=15, remove_responses=False, num_samples = 1):
  # initialize inputs and chatbot reponses
  input_text = ""
  chatbot_response = ""
  i=0
  oldtext=""
  temp=.8 # what does temp do?
  moving_window_used = False

  # Initialize lists to store the prompts and responses.
  client_prompts = list()
  therapist_responses = list()
  while input_text!="ABORT":
    i=i+1
    if i==1:
      print("Can you tell me what you want to talk about today? It helps if you say at least a few sentences for context.")
    input_text = input("\n")
    client_prompts.append(input_text) # adds the user input to prompts list

    if input_text=="ABORT":
      break
    
    if(remove_responses == False):
      # Filter the input text so that it fits into the moving window
      if(len(client_prompts) <= moving_window):
        combined_text = ""
        for j in range(len(client_prompts)):
          if j != 0:
            combined_text = combined_text + therapist_responses[j - 1]

          combined_text = combined_text + "CLIENT: " + client_prompts[j] + "\nTHERAPIST:" 
      else: 
        moving_window_used = True
        combined_text = ""
        prompts_subset = client_prompts[-moving_window:]
        responses_subset = therapist_responses[-moving_window + 1:]

        for j in range(len(prompts_subset)):
          if j != 0:
            combined_text = combined_text + responses_subset[j - 1]
          combined_text = combined_text + "CLIENT: " + prompts_subset[j] + "\nTHERAPIST:" 

    else: 
      combined_text = "Client: " + ' '.join(client_prompts[-moving_window:])

    print(f'Input text: {combined_text}')

    # Generate text from the combined input text
    single_text=gpt2.generate(sess,
                length=120,
                temperature=temp,
                prefix=combined_text,
                nsamples=num_samples,
                batch_size=1,
                return_as_list=True
                )

    # Determine where to split the response single text by checking if moving window is being used.
    if moving_window_used:
      split_index = moving_window 
    else:
      split_index = i 

    # Iteratively generate text until the response is greater than min_len_response 
    temp=.4
    while(len(single_text[0].split("THERAPIST:")[split_index].split("CLIENT:")[0])<min_len_response):
      temp=temp+.05
      single_text=gpt2.generate(sess,
                length=120,
                temperature=temp,
                prefix=combined_text,
                nsamples=num_samples,
                batch_size=1,
                return_as_list=True
                )
  
    responses = []
    responses_scores = []
    for k in range(len(single_text)):
      sample_response = single_text[k].split("THERAPIST:")[split_index].split("CLIENT:")[0]
      sample_score = calc_seance_score(sample_response)

      responses.append(sample_response)
      responses_scores.append(sample_score)

      print(f'sample response: {sample_response}')
      print(f'seance score: {sample_score}')
    
    response = single_text[0].split("THERAPIST:")[split_index].split("CLIENT:")[0]
    therapist_responses.append(response) 
    
    print("\nTherapist Bot:"+ response) #change 1 to i if the prior is re-insurted 

In [2]:
test_scores = [3,4,4]
print(test_scores.index(max(test_scores))) # change the scoring mechanism for time (heavily penalize week)

1


In [3]:
interact_with_bot(moving_window=3, min_len_response=16, remove_responses=True, num_samples = 20)  # insert max response length? # discard all therapy responses # merge  # add in all the potential resonses until the number of samples is set. # closest to client response (improves synchrony)

Can you tell me what you want to talk about today? It helps if you say at least a few sentences for context.

Hello I am feeling tired.
Input text: Client: Hello I am feeling tired.


NameError: ignored

In [0]:
def create_chat_outputs(prompts_list, num_responses = 100,moving_window=3, min_len_response=15, remove_responses=False, num_samples = 1):

  # Initialize an emtpy list for the future addition of dataframes.
  df_list = []

  # Iteratively enter each prompt until num_responses are produced by the nlp model. 
  for prompt in tqdm(prompts_list):
    print(prompt)
    output_dict = {}

    # initialize inputs and chatbot reponses
    input_text = ""
    chatbot_response = ""
    oldtext=""
    temp=.4 # what does temp do?
    moving_window_used = False
    output_list = []

    i=1
    input_text = "Client: " + prompt + "\nTHERAPIST"
    

    # Generate text from the combined input text
    while(len(output_list) < num_responses):
      output_dict['temp'] = temp
      generated_texts = gpt2.generate(sess,
                  length=120,
                  temperature=temp,
                  prefix=input_text,
                  nsamples=num_responses - len(output_list),
                  batch_size=1,
                  return_as_list=True
                  )
      
      temp += .05 # randomness parameter 

      # Determine where to split the response single text by checking if moving window is being used.
      if moving_window_used:
        split_index = moving_window 
      else:
        split_index = i

      responses = []

      # check if the outputs meet the requirements. 
      for k in range(len(generated_texts)):
        sample_response = generated_texts[k].split("THERAPIST:")[split_index].split("CLIENT:")[0].strip()
        if len(sample_response) > 1 and len(output_list) < num_responses:
          output_list.append(sample_response)
      
      output_set = set(output_list)
      output_list = list(output_set)

      print(f'For prompt number {len(df_list)}, {len(output_list)} outputs were created.')

    output_dict['input'] = prompt
    output_dict['output_list'] = output_list

    single_output_df = pd.DataFrame(output_dict)
    single_output_df.to_csv(f"drive/My Drive/5_datascience/2_context_chatbot/ouputs/prompt{len(df_list)}_output.csv")
    df_list.append(single_output_df)


  return(df_list)

    

In [0]:
all_prompts = pd.read_csv('drive/My Drive/5_datascience/2_context_chatbot/sample_prompts.csv')

In [0]:
all_outputs = create_chat_outputs(all_prompts['Prompt'])

  0%|          | 0/129 [00:00<?, ?it/s]

I suffer from debilitating social anxiety, which prevents me from having normal relationships. My negative thought is I will be alone forever.
For this prompt number 0, 43 outputs were created.
For this prompt number 0, 60 outputs were created.
For this prompt number 0, 79 outputs were created.
For this prompt number 0, 88 outputs were created.
For this prompt number 0, 93 outputs were created.
For this prompt number 0, 97 outputs were created.
For this prompt number 0, 99 outputs were created.
For this prompt number 0, 99 outputs were created.
For this prompt number 0, 99 outputs were created.


  1%|          | 1/129 [04:22<9:20:53, 262.92s/it]

For this prompt number 0, 100 outputs were created.
I'm freaking out studying for finals
For this prompt number 1, 20 outputs were created.
For this prompt number 1, 36 outputs were created.
For this prompt number 1, 51 outputs were created.
For this prompt number 1, 60 outputs were created.
For this prompt number 1, 70 outputs were created.
For this prompt number 1, 80 outputs were created.
For this prompt number 1, 87 outputs were created.
For this prompt number 1, 90 outputs were created.
For this prompt number 1, 95 outputs were created.
For this prompt number 1, 99 outputs were created.


  2%|▏         | 2/129 [11:36<11:04:39, 314.01s/it]

...

For this prompt number 3, 100 outputs were created.
I'm really overloaded with work and have a ton to do for tomorrow.
For this prompt number 4, 39 outputs were created.
For this prompt number 4, 58 outputs were created.
For this prompt number 4, 75 outputs were created.
For this prompt number 4, 84 outputs were created.
For this prompt number 4, 91 outputs were created.
For this prompt number 4, 96 outputs were created.


  4%|▍         | 5/129 [23:18<9:12:19, 267.25s/it]

For this prompt number 4, 100 outputs were created.
I'm feeling really stressed out all the time and I feel overwhelmed.
For this prompt number 5, 29 outputs were created.
For this prompt number 5, 46 outputs were created.
For this prompt number 5, 72 outputs were created.
For this prompt number 5, 81 outputs were created.
For this prompt number 5, 90 outputs were created.
For this prompt number 5, 96 outputs were created.
For this prompt number 5, 99 outputs were created.


  5%|▍         | 6/129 [29:06<9:57:31, 291.48s/it]

For this prompt number 5, 100 outputs were created.
I feel anxious


In [0]:
test_output[0].to_csv('test.csv')